In [1]:
import os
import re

In [2]:
files = []
directory = "../results/experiment_outputs/20251004T050412Z/0e08ff6131ec444a9dd2c11f735ce675"

In [3]:
def list_top_level_subdirectories(root_dir):
    # List only the top-level subdirectories
    for entry in os.listdir(root_dir):
        full_path = os.path.join(root_dir, entry)
        if os.path.isdir(full_path):
            files.append(entry)

In [4]:
def has_user_patches_subdir(folder_path):
    return os.path.isdir(os.path.join(folder_path, 'user_patches'))

In [5]:
# Ask user for a directory path
if os.path.isdir(directory):
    # print(f"Subdirectories inside '{directory}':\n")
    list_top_level_subdirectories(directory)
else:
    print("Invalid directory path.")

In [6]:
def get_error_word_message_from_content(filepath):
    FILE_LINE_RE = re.compile(r'^\s*File "([^"]+)", line (\d+)(?:, in (.+))?')
    ERROR_LINE_RE = re.compile(r'^\s*(\w+(?:Error|Exception))(?:\s*:\s*(.*))?$')
    SORRY_LINE_RE = re.compile(r'^\s*Sorry:\s*(\w+(?:Error|Exception))\s*:\s*(.*?)\s*\(([^,]+),\s*line\s*(\d+)\)\s*$')
    error_word = []
    messages = []
    last_file = None
    last_line = None
    last_ctx = None
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            s = line.rstrip("\n")
            m_file = FILE_LINE_RE.match(s)
            if m_file:
                last_file = m_file.group(1)
                try:
                    last_line = int(m_file.group(2))
                except ValueError:
                    last_line = None
                last_ctx = m_file.group(3)
                continue
            m_err = ERROR_LINE_RE.match(s)
            if m_err:
                error_word.append(m_err.group(1))
                messages.append((m_err.group(2) or "").strip())
                last_file = None
                last_line = None
                last_ctx = None
                break
            # 1) Handle 'Sorry:' one-liners immediately
            m_sorry = SORRY_LINE_RE.match(line)
            if m_sorry:
                error_word.append(m_sorry.group(1))
                messages.append((m_sorry.group(2) or "").strip())
                last_file = None
                last_line = None
                last_ctx = None
                break
            
    error_word = error_word[0] if len(error_word) > 0 else None
    message = messages[0] if len(messages) > 0 else None
    # print(f"File: {filepath}, Error Word: {error_word}, Message: {message}")
    return error_word, message


In [7]:
def search_success_equal_line(filepath,content):
    error_word, error_message = get_error_word_message_from_content(filepath)
    # print(error_word, error_message)
    return (error_word, error_message)

In [8]:
def get_error_description_from_content(content, error_word):
    import re
    lines = content.splitlines()
    description_lines = []
    for line in lines:
        description_lines.append(line)
        if error_word and error_word in line:
            break
    description = ' '.join(description_lines)
    description = re.sub(r'\s+', ' ', description)
    return description.strip()

In [9]:
def get_error_message_after_word(content, error_word):
    import re
    for line in content.splitlines():
        if error_word and error_word in line:
            idx = line.find(error_word)
            if idx != -1:
                # Remove colon and whitespace after error word
                after = line[idx + len(error_word):]
                after = re.sub(r'^\s*:\s*', '', after)
                return after.strip()
    return None

In [10]:
def get_module_lines(content):
    module_lines = []
    for line in content.splitlines():
        if '***<module>' in line:
            module_lines.append(line)
    return module_lines

In [11]:
def clean_error_message(error_message):
    import re
    if not error_message:
        return None
    # Convert to lowercase
    cleaned_message = error_message.lower()
    # Remove leading and trailing whitespace
    cleaned_message = cleaned_message.strip()
    # Remove contents inside parentheses (including parentheses)
    cleaned_message = re.sub(r'\(.*?\)', '', cleaned_message)
    cleaned_message = cleaned_message.strip()
    return cleaned_message

In [47]:
import pandas as pd

dataframe = pd.DataFrame()
def parse_highest_equivalence_report(subdir):
    file_name = subdir
    # print(f"Processing file: {file_name}")
    subdir_path = os.path.join(os.path.join(directory, subdir))
    pattern = re.compile(r"syntax_failed_repaired_(\d+)_error\.txt")
    max_num = -1
    max_file = None
    
    if not os.path.isdir(subdir_path):
        print(f"Directory not found: {subdir_path}")
        return
    for fname in os.listdir(subdir_path):
        match = pattern.match(fname)
        if match:
            num = int(match.group(1))
            if num > max_num:
                max_num = num
                max_file = fname
    if max_file:
        with open(os.path.join(subdir_path, max_file), "r") as f:
            content = f.read()
        error_word, error_message = search_success_equal_line(os.path.join(subdir_path, max_file), content)
        # equivalence = not (error_word or has_module_line)
        # if error_word:
        #     error_type = "syntactic_error"
        # elif has_module_line and not error_word:
        #     error_type = "semantic_error"
        error_description = None
        if error_word:
            error_description = get_error_description_from_content(content, error_word)
            # if error_description:
            #     error_message = get_error_message_after_word(content, error_word)
        parsing_metadata = {"file_hash": file_name, "syntactic_error_word": error_word, "syntactic_error_message": error_message if error_message else None,  "precessed_error_message": clean_error_message (error_message) if error_message else None, "syntactic_error_description": error_description if error_word else None}
        new_df = pd.DataFrame([parsing_metadata])
        global dataframe
        dataframe = pd.concat([dataframe, new_df], ignore_index=True)

In [48]:
for file in files:
    # print(f"Processing file: {file}")
    parse_highest_equivalence_report(file)

In [49]:
dataframe

,file_hash,syntactic_error_word,syntactic_error_message,precessed_error_message,syntactic_error_description
0,d3204438087d7d924f50237ca1438a01c5d8bf5e26bb23...,SyntaxError,unmatched ')',unmatched ')',"File ""results/experiment_outputs/20251004T0504..."
1,f131e8b1b384f595ae4976583b055a565560f5f1d70164...,SyntaxError,closing parenthesis ']' does not match opening...,closing parenthesis ']' does not match opening...,"File ""results/experiment_outputs/20251004T0504..."
2,410c8bffc380e1f887d96db79780392e309662f3b6e135...,SyntaxError,invalid syntax. Perhaps you forgot a comma?,invalid syntax. perhaps you forgot a comma?,"File ""results/experiment_outputs/20251003T1658..."
3,46e112f054ecc3b819ca39178321dc01d23be5e49c2151...,SyntaxError,too many nested parentheses,too many nested parentheses,"File ""results/experiment_outputs/20251003T0117..."
4,71ddfa42351aae24b5c2bfe98c77117fd09f683ac3da97...,SyntaxError,closing parenthesis ']' does not match opening...,closing parenthesis ']' does not match opening...,"File ""results/experiment_outputs/20251004T0504..."
...,...,...,...,...,...
1226,0d0e18982e4642228749e3cb47d075d43b99118821a488...,SyntaxError,from __future__ imports must occur at the begi...,from __future__ imports must occur at the begi...,"File ""results/experiment_outputs/20251003T1658..."
1227,d9727c6061e03c321a3d3df93244e8eb2d2913fe7b8f0c...,SyntaxError,f-string: single '}' is not allowed,f-string: single '}' is not allowed,"File ""results/experiment_outputs/20251004T0504..."
1228,4dd96e1585dad60e26a6e0efac04a9a8f432a01cd6f258...,SyntaxError,'(' was never closed,'(' was never closed,"File ""results/experiment_outputs/20251003T1658..."
1229,c34fdfcf7b00ea897f2e3142512aea6835a7e1f72890d9...,IndentationError,unexpected indent,unexpected indent,Sorry: IndentationError: unexpected indent (sy...


In [50]:
replacement_values = {
    'syntactic_error_word': "ValueError",
    'syntactic_error_message': "version None is invalid",
    'precessed_error_message': "version none is invalid",
    'syntactic_error_description': "ValueError: version None is invalid"
}

In [51]:
# Replace None with the specific strings in the relevant columns
for column, replacement in replacement_values.items():
    dataframe[column].fillna(replacement, inplace=True)

/tmp/ipykernel_8479/1316908363.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataframe[column].fillna(replacement, inplace=True)


In [52]:
# Replace None with 'N/A' in the dataframe
dataframe = dataframe.fillna("None")

# Example usage after replacement
print(dataframe)

                                              file_hash syntactic_error_word  \
0     d3204438087d7d924f50237ca1438a01c5d8bf5e26bb23...          SyntaxError   
1     f131e8b1b384f595ae4976583b055a565560f5f1d70164...          SyntaxError   
2     410c8bffc380e1f887d96db79780392e309662f3b6e135...          SyntaxError   
3     46e112f054ecc3b819ca39178321dc01d23be5e49c2151...          SyntaxError   
4     71ddfa42351aae24b5c2bfe98c77117fd09f683ac3da97...          SyntaxError   
...                                                 ...                  ...   
1226  0d0e18982e4642228749e3cb47d075d43b99118821a488...          SyntaxError   
1227  d9727c6061e03c321a3d3df93244e8eb2d2913fe7b8f0c...          SyntaxError   
1228  4dd96e1585dad60e26a6e0efac04a9a8f432a01cd6f258...          SyntaxError   
1229  c34fdfcf7b00ea897f2e3142512aea6835a7e1f72890d9...     IndentationError   
1230  9555ab51eeaedf714072d0e831223f417232150d9b50fb...           ValueError   

                                syntact

In [53]:
def summarize_dataframe(df):
    if df.empty:
        print("The dataframe is empty. No summaries to display.")
        return

    # Group by error type
    grouped = df.groupby("syntactic_error_word").agg(
        total_data=("file_hash", "count"),
    )

    print("Summary by Error Type:")
    print(grouped)

# Example usage
summarize_dataframe(dataframe)

Summary by Error Type:
                      total_data
syntactic_error_word            
IndentationError              62
SyntaxError                 1028
ValueError                   141


In [54]:
def save_dataframe_to_csv(df, file_path):
    try:
        df.to_csv(file_path, index=False)
        print(f"Dataframe successfully saved to {file_path}")
    except Exception as e:
        print(f"An error occurred while saving the dataframe: {e}")

In [62]:
def save_dataframe_to_excel(df, file_path):
    try:
        df.to_excel(file_path, index=False)
        print(f"Dataframe successfully saved to {file_path}")
    except Exception as e:
        print(f"An error occurred while saving the dataframe: {e}")

In [64]:
save_dataframe_to_excel(dataframe, "dataset_summary_failed_case.xlsx")

Dataframe successfully saved to dataset_summary_failed_case.xlsx


In [60]:
dataframe[dataframe["syntactic_error_message"] == "invalid syntax"]

,file_hash,syntactic_error_word,syntactic_error_message,precessed_error_message,syntactic_error_description
5,b4643d5ad41eb5bd2698b2e41e6c53c17a2ea51fb7dea2...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251004T0504..."
14,ca7d7759d86ee4c53375e931ccc31f6a58c54edd2ca5b6...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251004T0504..."
16,8c2b12c78c8066218f79b33f1607939506b1434175d297...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251004T0046..."
18,36e448d4f93337ef04510146d26cb057ce4be8204a6054...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251004T0504..."
24,62cb8090ed1eca4252c3c2cc0fe1abaebabfbccdacc975...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251003T0117..."
...,...,...,...,...,...
1211,bb66bf755f5d0e8df38449123034a80ee5f0d08b8c6637...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251004T0504..."
1212,a3da0b0a572012c2f92044da0c1cf187a7d030c6e423f7...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251003T1658..."
1217,b8e5d9d02baec7711a620a64e164c2df55480248d95e06...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251003T0117..."
1220,00a7db70a07968ecd1600e639370e23ed90d198c511821...,SyntaxError,invalid syntax,invalid syntax,"File ""results/experiment_outputs/20251003T1658..."
